In [28]:
from keras.models import load_model
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras import layers
from tensorflow import keras

In [29]:
class VAE(keras.Model):
    def __init__(self, encoder, decoder, **kwargs):
        super(VAE, self).__init__(**kwargs)
        self.encoder = encoder
        self.decoder = decoder
        self.total_loss_tracker = keras.metrics.Mean(name="total_loss")
        self.reconstruction_loss_tracker = keras.metrics.Mean(
            name="reconstruction_loss"
        )
        self.kl_loss_tracker = keras.metrics.Mean(name="kl_loss")
        #self.beta_coefficient = beta_coefficient
    
    def call(self, inputs):
        x = self.encoder(inputs)[2]
        return self.decoder(x)
    @property
    def metrics(self):
        return [
            self.total_loss_tracker,
            self.reconstruction_loss_tracker,
            self.kl_loss_tracker,
        ]

    def train_step(self, data):
        with tf.GradientTape() as tape:
            z_mean, z_log_var, z = self.encoder(data)
            reconstruction = self.decoder(z)
            reconstruction_loss = tf.reduce_sum(
                    keras.losses.MSE(data, reconstruction), axis=(1, 2) # mod
                )
           # )
            kl_loss = -0.5 * (1 + z_log_var - tf.square(z_mean) - tf.exp(z_log_var))
            kl_loss = tf.reduce_mean(tf.reduce_sum(kl_loss, axis=1))
            total_loss = reconstruction_loss + kl_loss
        grads = tape.gradient(total_loss, self.trainable_weights)
        self.optimizer.apply_gradients(zip(grads, self.trainable_weights))
        self.total_loss_tracker.update_state(total_loss)
        self.reconstruction_loss_tracker.update_state(reconstruction_loss)
        self.kl_loss_tracker.update_state(kl_loss)
        return {
            "loss": self.total_loss_tracker.result(),
            "reconstruction_loss": self.reconstruction_loss_tracker.result(),
            "kl_loss": self.kl_loss_tracker.result(),
        }

In [30]:

vae_encoder = load_model('models/vae_encoder.h5')
vae_decoder = load_model('models/vae_decoder.h5')
vae = VAE(encoder=vae_encoder, decoder=vae_decoder)
vae(np.zeros((1,50,50,3)))

<tf.Tensor: shape=(1, 50, 50, 3), dtype=float32, numpy=
array([[[[0.1301477 , 0.        , 0.        ],
         [0.08753122, 0.04897526, 0.2308377 ],
         [0.08454477, 0.15004405, 0.31561223],
         ...,
         [0.02351195, 0.03280998, 0.        ],
         [0.        , 0.10371088, 0.        ],
         [0.        , 0.07004683, 0.        ]],

        [[0.32667994, 0.20192355, 0.        ],
         [0.33602655, 0.14119574, 0.145914  ],
         [0.23726887, 0.19451548, 0.32976425],
         ...,
         [0.16604294, 0.00813007, 0.        ],
         [0.        , 0.06600943, 0.03235287],
         [0.        , 0.08069658, 0.        ]],

        [[0.30490837, 0.16846782, 0.04510091],
         [0.22380856, 0.34690136, 0.09409196],
         [0.34124374, 0.41359252, 0.03976146],
         ...,
         [0.21198794, 0.03353767, 0.03062569],
         [0.        , 0.00796714, 0.21419299],
         [0.        , 0.04374745, 0.09003568]],

        ...,

        [[0.10269634, 0.21908036, 0.

In [31]:
vae.compile(optimizer='RMSprop')
vae.load_weights('weights/vae.h5')
model = vae_decoder

IndexError: list index out of range

In [ ]:
import cv2

test = cv2.imread('datasets/breast-histopathology/IDC_regular_ps50_idx5/13689/1/13689_idx5_x801_y1501_class1.png', cv2.IMREAD_COLOR)

In [ ]:
test = test / 255
test.shape

In [ ]:
# We preprocess the image into a 4D tensor
from keras.preprocessing import image
import numpy as np

img = cv2.imread('datasets/breast-histopathology/IDC_regular_ps50_idx5/13689/1/13689_idx5_x801_y1501_class1.png', cv2.IMREAD_COLOR)
img_tensor = image.img_to_array(img)
img_tensor = np.expand_dims(img_tensor, axis=0)
# Remember that the model was trained on inputs
# that were preprocessed in the following way:
img_tensor /= 255.

# Its shape is (1, 150, 150, 3)
print(img_tensor.shape)

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

plt.imshow(img_tensor[0])
plt.show()

In [ ]:
from keras import models

# Extracts the outputs of the top 8 layers:
import tensorflow as tf

layer_outputs = []
layer_names = []
for layer in model.layers[1:]:
    
    try: 
        layer_outputs.append(layer.get_output_at(1))
        layer_names.append(layer.name)
    
    except:
        layer_outputs.append(layer.output)
        layer_names.append(layer.name)

# Creates a model that will return these outputs, given the model input:
activation_model = models.Model(inputs=model.input, outputs=layer_outputs)
activation_model

In [ ]:
layer_names

In [ ]:
model.get_layer('up_block6_conv1')

In [ ]:
model.get_layer('up_block6_conv1')

In [ ]:
# This will return a list of 5 Numpy arrays:
# one array per layer activation
activations = activation_model.predict(vae_encoder(img_tensor)[2])
activations

In [ ]:
print(len(activations))
print(len(layer_names))

In [ ]:
first_layer_activation = activations[6]
print(first_layer_activation.shape)
first_layer_activation.shape[3]

In [ ]:
import matplotlib.pyplot as plt

plt.matshow(first_layer_activation[0, :, :, 1], cmap='viridis')
plt.show()

In [ ]:
plt.matshow(first_layer_activation[0,:, :, 63], cmap='viridis')
plt.show()

In [ ]:
activ = first_layer_activation[0,:, :, 5]

In [ ]:
f_min, f_max = activ.min(), activ.max()
filters = (activ - f_min) / (f_max - f_min) 
filters.max()

In [ ]:
activation_layer = first_layer_activation
n = math.floor(np.sqrt(first_layer_activation.shape[3]))
int(n + 0.5) ** 2 

In [ ]:
m = n

In [ ]:
filter_counter = 0
for i in range(n):
    for j in range(m):
     

       # ax[i, j].imshow(activation_layer[0,:, :, filter_counter], cmap='viridis')
        filter_counter += 1
        print(i)
        print(j)
        print(filter_counter)

In [ ]:
 (activation_layer.shape[3] - 1)

In [ ]:
import math
def plot_filters(activation_layer, layer_name, counter):

    n = math.floor(np.sqrt(activation_layer.shape[3]))

    if int(n + 0.5) ** 2 == activation_layer.shape[3]:

        m = n
    else:
        m = math.floor(activation_layer.shape[3] / n)

    
    if n == 1:
        fig, ax = plt.subplots(1, 3, sharex='col', sharey='row',figsize=(15, 15))
        fig.suptitle(layer_name)
        for i in range(3):
            ax[i].imshow(activation_layer[0,:, :, i], cmap='viridis')
        fig.savefig('img/activations/vae/{}_activations_{}.png'.format(counter, layer_name))
        return None   

    fig, ax = plt.subplots(n, m, sharex='col', sharey='row',figsize=(15, 15))
    fig.suptitle(layer_name)
    
 

    filter_counter = 0
    for i in range(n):
        for j in range(m):
            ax[i, j].imshow(activation_layer[0,:, :, filter_counter], cmap='viridis')
            filter_counter += 1
            if filter_counter == (activation_layer.shape[3] ):
                break
    
    fig.savefig('img/activations/vae/{}_activations_{}.png'.format(counter, layer_name))
    return None

In [ ]:
for activation, name in zip(activations[1:], layer_names[1:]):
    print(name)
    print(activation.shape)
    

In [ ]:
for counter, (activation, name) in enumerate(zip(activations[1:], layer_names[1:])):

    plot_filters(activation, name, counter)

In [ ]:
for layer in net.layers:
    print(layer.name)

In [ ]:
net.summary()

In [ ]:
net.get_layer('sequential_14')